<a href="https://colab.research.google.com/github/monicaindrawan/kpop_survey_analysis/blob/main/kpop_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect google colab to google drive
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# cd = change directory
%cd "/content/gdrive/MyDrive/Colab Notebooks"

/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
# install pandas
!pip install pandas

In [ ]:
# import pandas in notebook
import pandas as pd
# read and print dataframe
df = pd.read_excel("KPOP DATA.xlsx")
df['Num'] = [i for i in range(len(df))]
col = list(df.columns.values)
df = df[[col[-1]] + col[0:23]]
df

,Num,Timestamp,Which is your favourite K-Pop group?,Is K-Pop popular in your country?,Why do you like K-Pop?,When did you start listening to K-Pop?,Do you listen to K-Pop male groups more than girl group songs?,How many hours do you spend listening to K-Pop?,Do you watch K-Pop Youtube videos?,"If you do watch K-Pop music videos, how long do you spend watching them?",Would you say that you are obsessed with K-Pop?,What do you do to keep up with K-Pop news?,Has K-Pop led you to pursue any of the following?,Has consuming K-Pop taken away most of your time?,Has consuming K-Pop affected your personal life in the following ways?,Have you experienced any positive effects after consuming K-Pop?,"On an average, in one year, how much do you spend on K-Pop merchandise?","If you do spend money on K-Pop merchandise, where do you get money from?",Do you attend K-Pop concerts?,What is the craziest thing you have done in your pursuit and love for K-Pop?,How old are you?,Which country are you from?,What is your profession?,What is your gender?
0,0,2019/02/23 4:04:31 AM GMT+5:30,BTS,Its gaining popularity,Its different from the usual music. I like som...,1 -2 years ago,"Its about the music, not the gender",5 or more,Yes,2-3 hours,Not sure,Subscribe to K-Pop news sites;Subscribe to You...,Learning about Korean/Asian culture;Pursue dan...,No,Led to you being cajoled (made fun of) by clas...,Reduced stress/anxiety/depression,50 - 100 $,I have a full - time job,Yes,Missed days of work and school to attend concerts,15 - 20 years,A country in the UK,Sales,Female
1,1,2019/02/23 4:04:38 AM GMT+5:30,BTS,Its gaining popularity,Its different from the usual music. I like som...,3-4 years ago,I listen to both,2-4 hours,Yes,2-3 hours,Yes,Subscribe to K-Pop news sites;Subscribe to You...,Learning music - either an instrument or singi...,Yes,Led to you being cajoled (made fun of) by clas...,Made friends who are like-minded,Below 50$,Working part-time,"I want to, but the K-Pop groups don't come to ...",Listened to K-Pop without studying or working ...,15 - 20 years,A country in the UK,Student,Female
2,2,2019/02/23 4:04:56 AM GMT+5:30,BTS;MONSTA X,Its gaining popularity,Its different from the usual music. I like som...,More than 4 years ago,"Its about the music, not the gender",2-4 hours,Yes,2-3 hours,No,Subscribe to K-Pop news sites;Join K-pop socia...,Learning about Korean/Asian culture;Learn abou...,No,Led to you being cajoled (made fun of) by clas...,Made friends who are like-minded,50 - 100 $,Working part-time,Yes,Nothing as such,15 - 20 years,Other European countries,Student,Female
3,3,2019/02/23 4:04:58 AM GMT+5:30,BTS,Its gaining popularity,Its different from the usual music. I like som...,3-4 years ago,I listen to both,5 or more,Yes,3-4 hours,No,Subscribe to K-Pop news sites;Subscribe to You...,Learning about Korean/Asian culture;Visit Korea,No,Led to you being cajoled (made fun of) by clas...,Reduced stress/anxiety/depression,50 - 100 $,Working part-time,Yes,Nothing as such,21 - 26 years,USA,Student,Female
4,4,2019/02/23 4:05:12 AM GMT+5:30,BTS,Yes,Its different from the usual music. I like som...,3-4 years ago,Yes,5 or more,Yes,2-3 hours,Not sure,Subscribe to Youtube channels,Learning about Korean/Asian culture,Yes,Led to you being cajoled (made fun of) by clas...,Reduced stress/anxiety/depression,100 - 200$,Parents,"I want to, but the K-Pop groups don't come to ...",Nothing as such,15 - 20 years,USA,Student,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,235,2019/03/15 5:01:53 AM GMT+5:30,MONSTA X,Its gaining popularity,Its different from the usual music. I like som...,3-4 years ago,I listen to both,5 or more,Yes,3-4 hours,Not sure,Subscribe to Youtube channels,Pursue dancing;Learn about Korean fashion/makeup,Not sure,Reduced the amount of sleep you get,Reduced stress/anxiety/depression,I don't spend anything on merchandise,I donâ€™t,"I want to, but the K-Pop groups don't come to ...",Not

In [ ]:
df = df.drop(columns=['Timestamp'])
df = df.rename(columns={
        'Which is your favourite K-Pop group?':'fav_group',
        'Is K-Pop popular in your country? ':'pop_by_country', 
        'Why do you like K-Pop?':"reason",
        'When did you start listening to K-Pop?':"first_year",
        'Do you listen to K-Pop male groups more than girl group songs?':"kpop_gender_pref",
        'How many hours do you spend listening to K-Pop?':"daily_music_hr",
        'Do you watch K-Pop Youtube videos?':"watch_mv_yn",
        'If you do watch K-Pop music videos, how long do you spend watching them?':"daily_mv_hr",
        'Would you say that you are obsessed with K-Pop?':"obsessed_yn",
        'What do you do to keep up with K-Pop news?':"kpop_news",
        'Has K-Pop led you to pursue any of the following?':"pursuit",
        'Has consuming K-Pop taken away most of your time?':"time_consume_yn",
        'Has consuming K-Pop affected your personal life in the following ways?':"life_changing",
        'Have you experienced any positive effects after consuming K-Pop? ':"pos_eff",
        'On an average, in one year, how much do you spend on K-Pop merchandise?':"merch_spent_yr",
        'If you do spend money on K-Pop merchandise, where do you get money from?':"money_obtain",
        'Do you attend K-Pop concerts?':"concert_yn",
        'What is the craziest thing you have done in your pursuit and love for K-Pop?':"crazy_thing",
        'How old are you?':"age", 
        'Which country are you from?':"domicile",
        'What is your profession?':"job",
         'What is your gender?':"gender"
         })

In [ ]:
df['life_changing'].fillna("none", inplace=True)
df['money_obtain'].fillna("none", inplace=True)
df.isnull().sum()

Num                 0
fav_group           0
pop_by_country      0
reason              0
first_year          0
kpop_gender_pref    0
daily_music_hr      0
watch_mv_yn         0
daily_mv_hr         5
obsessed_yn         0
kpop_news           0
pursuit             0
time_consume_yn     0
life_changing       0
pos_eff             0
merch_spent_yr      0
money_obtain        0
concert_yn          0
crazy_thing         0
age                 0
domicile            0
job                 0
gender              0
dtype: int64

In [ ]:
mymap = {'1 hour':1, '2-3 hours':2.5, '3-4 hours':3.5, 'More than four hours':4.5}
df = df.applymap(lambda s: mymap.get(s) if s in mymap else s)
df["daily_mv_hr"].fillna(df["daily_mv_hr"].mode().values[0] , inplace = True)
df["daily_mv_hr"]= df["daily_mv_hr"].replace(1.0, "1 hour").replace(2.5,"2-3 hours").replace(3.5,"3-4 hours").replace(4.5,"More than four hours")
df.groupby(["daily_mv_hr"]).count()

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,daily_music_hr,watch_mv_yn,obsessed_yn,kpop_news,pursuit,time_consume_yn,life_changing,pos_eff,merch_spent_yr,money_obtain,concert_yn,crazy_thing,age,domicile,job,gender
daily_mv_hr,,,,,,,,,,,,,,,,,,,,,,
1 hour,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123,123
2-3 hours,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84
3-4 hours,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20
More than four hours,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13


In [ ]:
df["daily_music_hr"] = df["daily_music_hr"].astype(str).replace("2020-07-24 00:00:00", "Others")
all_list = ["all", "whole", "everyday", "hours on end", "most", "every hour"]
everytime_list = ["any", "chance", "anytime", "whenever"]
mood_list = ["depend","mood"]
ans1_list = ["2-4 hours"]
ans2_list = ["5 or more", "half of my day"]
ans3_list = ["less than an hour"]

def music_hour(x):
  if any(keyword in x.lower() for keyword in everytime_list):
    return "Whenever listen to music"
  elif any(keyword in x.lower() for keyword in mood_list):
    return "Depend on the mood"
  elif any(keyword in x.lower() for keyword in all_list):
    return "24/7"
  elif any(keyword in x.lower() for keyword in ans1_list):
    return "2-4 hours"
  elif any(keyword in x.lower() for keyword in ans2_list):
    return "5 or more"
  elif any(keyword in x.lower() for keyword in ans3_list):
    return "Less than an hours"
  else:
    return "Others"

df["daily_music_hr"] = df["daily_music_hr"].apply(lambda x: music_hour(x))

df.groupby(["daily_music_hr"]).count()

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,time_consume_yn,life_changing,pos_eff,merch_spent_yr,money_obtain,concert_yn,crazy_thing,age,domicile,job,gender
daily_music_hr,,,,,,,,,,,,,,,,,,,,,,
2-4 hours,106,106,106,106,106,106,106,106,106,106,106,106,106,106,106,106,106,106,106,106,106,106
24/7,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
5 or more,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
Depend on the mood,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Less than an hours,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
Others,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Whenever listen to music,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


In [ ]:
def clean_first_year(x):
    if re.search('[0-9]+', x) != None:
      num = int(re.search('[0-9]+', x).group())
      if num > 1000 and num % 2 == 0:
        return str(2020-num-1) + "-" + str(2020-num) + " years ago"
      elif num < 1000 and num % 2 == 0:
        return str(num-1) + "-" + str(num) + " years ago"
      elif num > 1000 and num % 2 != 0:
        return str(2020-num) + "-" + str(2020-num+1) + " years ago"
      elif num < 1000 and num % 2 != 0:
        return str(num) + "-" + str(num+1) + " years ago"
    else:
      return x

import re 
df["first_year"] = df["first_year"].apply(lambda x: clean_first_year(x))
df["first_year"]= df["first_year"].replace("Iâ€™ve been listening to it all my life basically, my cousins are Asian, and Iâ€™m mexican, so people think Iâ€™m just a koreaboo, when in reality Iâ€™ve only ever listened to korean and Chinese music", 
                                           "Since childhood")
df.groupby(["first_year"]).count()

,Num,fav_group,pop_by_country,reason,kpop_gender_pref,daily_music_hr,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,time_consume_yn,life_changing,pos_eff,merch_spent_yr,money_obtain,concert_yn,crazy_thing,age,domicile,job,gender
first_year,,,,,,,,,,,,,,,,,,,,,,
1-2 years ago,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66
13-14 years ago,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3-4 years ago,158,158,158,158,158,158,158,158,158,158,158,158,158,158,158,158,158,158,158,158,158,158
5-6 years ago,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7-8 years ago,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
9-10 years ago,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Less than a year ago,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
Since childhood,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
df["money_obtain"] = df.apply(lambda x: "-" if x["merch_spent_yr"] == "I don't spend anything on merchandise" else x["money_obtain"], axis = 1)

set1 = ["-"]
set2 = ["Parents","parents"]
set3 = ["just","save","Savings","account","birthday", "christmas","presents"]
set4 = ["Work","work","Chores","help"]
set5 = ["Borrow", "Aid"]
set6 = ["full - time job"]

df["money_obtain"] = df["money_obtain"].str.replace("i donâ€™t spend money on kpop merchandise.","-")
def clean_money_obtain(x):
    if any(keyword in x for keyword in set6):
      return "Have full-time job"
    elif any(keyword in x for keyword in set5):
      return "Financial aid"
    elif any(keyword in x for keyword in set4):
      return "Working part-time & Doing chores"
    elif any(keyword in x for keyword in set3):
      return "Savings & Special occasion"
    elif any(keyword in x for keyword in set2):
      return "Parents"
    elif any(keyword in x for keyword in set1):
      return "-"
    else:
      return "Others"

df["money_obtain"] = df["money_obtain"].apply(lambda x: clean_money_obtain(x))

df.groupby(["money_obtain"]).count()

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,daily_music_hr,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,time_consume_yn,life_changing,pos_eff,merch_spent_yr,concert_yn,crazy_thing,age,domicile,job,gender
money_obtain,,,,,,,,,,,,,,,,,,,,,,
-,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50
Financial aid,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Have full-time job,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20
Others,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
Parents,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81
Savings & Special occasion,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
Working part-time & Doing chores,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68


In [ ]:
reason_list = []
tmp = df['reason'].to_list()
for reasons in tmp:
	for reason in reasons.split(';'):
		reason_list.append(reason)

reason_count = pd.Series(reason_list).value_counts()
top_reasons_list = list(reason_count[reason_count>1].index)
all_list = ["all, everything"]

reason_df = pd.DataFrame()
reason_df['Num'] = df['Num']
for reason in top_reasons_list:
    reason_df[reason] = df['reason'].apply(lambda x: 1 if (reason in x) or (any(keyword in x.lower() for keyword in all_list)) else 0)

reason_dict = { 
"Uplifting music and meaningful lyrics" : ["catchy", "insight", "meaning", "meaningful", "uplifting", "inspirational"],
"Loyal to fans" : ["fandom", "fan", "loyal", "relationship"],
"Reduced anxiety and bring happiness" : ["struggle", "tough", "depression", "suicidal", "happy", "happier", "fun"],
"Personality, talent and work ethic of the idols" : ["personality", "creativity", "personalities",  "hard", "work", "talent"],
"Study korean culture" : ["culture", "study", "learn"]
}
for reason in reason_dict.keys():
       reason_df[reason] = df['reason'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in reason_dict[reason]) else 0)

def other_reason(x):
    check_list = []
    for tmp in x.split(";"):
      if (all(all(keyword not in tmp.lower() for keyword in reason_dict[reason]) for reason in reason_dict.keys())) and \
      (all(reason not in tmp for reason in top_reasons_list)) and (all(value not in tmp.lower() for value in all_list)):
        check_list.append(True)
      else:
        check_list.append(False)
    if any(check_list):
      return 1
    else:
      return 0

reason_df["Others"] = df['reason'].apply(lambda x: other_reason(x))
reason_df.to_csv("reason.csv", index = False)

In [ ]:
news_list = []
tmp = df["kpop_news"].to_list()
for news in tmp:
  for new in news.split(";"):
    news_list.append(new)

news_count = pd.Series(news_list).value_counts()
top_kpopnews_list = list(news_count[news_count>10].index)
youtube_list = ["youtube"]
website_list = ["soompi", "allkpop"]
group_list = ["amino","reddit"]

kpopnews_df = pd.DataFrame()
kpopnews_df['Num'] = df['Num']
for kpopnews in top_kpopnews_list:
  if kpopnews == "Subscribe to Youtube channels":
    kpopnews_df[kpopnews] = df['kpop_news'].apply(lambda x: 1 if (kpopnews in x) or any(keyword in x.lower() for keyword in youtube_list) else 0)
  elif kpopnews == "Join K-pop social media groups":
    kpopnews_df[kpopnews] = df['kpop_news'].apply(lambda x: 1 if (kpopnews in x) or any(keyword in x.lower() for keyword in group_list) else 0)
  elif kpopnews == "Subscribe to K-Pop news sites":
    kpopnews_df[kpopnews] = df['kpop_news'].apply(lambda x: 1 if (kpopnews in x) or any(keyword in x.lower() for keyword in website_list) else 0)

news_dict = {
"Twitter" : ["twitter", "tweet"],
"Instagram" : ["instagram", "feeds"],
"Soompi / Allkpop" : ["soompi"],
"Follow idols' or fanbase social media accounts" : ["follow", "fan"]
}

for news in news_dict.keys():
  kpopnews_df[news] = df["kpop_news"].apply(lambda x: 1 if any(keyword in x.lower() for keyword in news_dict[news]) else 0)

def other_news(x):
    check_list = []
    for tmp in x.split(';'):
        if all(all(keyword not in tmp.lower() for keyword in news_dict[news]) for news in news_dict.keys()) and \
           all(keyword not in tmp for keyword in top_kpopnews_list) and \
           all(keyword not in tmp.lower() for keyword in (youtube_list+website_list+group_list)):
            check_list.append(True)
        else:
            check_list.append(False)
    
    if any(check_list):
        return 1
    else:
        return 0

kpopnews_df["Others"] = df["kpop_news"].apply(lambda x: other_news(x))
kpopnews_df.to_csv("kpop_news.csv", index = False)

In [ ]:
group_dict = {
"BTS" : ["BTS"],
"MONSTA X" : ["MONSTAX", "MONSTA X"],
"EXO" : ["EXO"],
"BLACKPINK" : ["BLACKPINK", "BLACK PINK"],
"STRAY KIDS" : ["STRAY KIDS", "STRAYKIDS"],
"NCT" : ["NCT"],
"GOT7" : ["GOT7", "GOT 7"],
"SEVENTEEN" : ["SEVENTEEN", "SEVENTERN", "SEVEN TEEN"],
"ATEEZ" : ["ATEEZ"],
"SHINEE" : ["SHINEE"]
}

import re
favgroup_df = pd.DataFrame()
favgroup_df['Num'] = df['Num']
for tmp in group_dict.keys():
    favgroup_df[tmp] = df["fav_group"].apply(lambda x: 1 if any(keyword in x.upper() for keyword in group_dict[tmp]) else 0)

def other_group(x):
    check_list = []
    for tmp in re.split(';|; | & |, |,| / | and |,and ',x):
        if all(all(keyword not in tmp.upper() for keyword in group_dict[group]) for group in group_dict.keys()):
            check_list.append(True)
        else:
            check_list.append(False)
    
    if any(check_list):
        return 1
    else:
        return 0

favgroup_df["Others"] = df["fav_group"].apply(lambda x: other_group(x))

favgroup_df.to_csv("fav_group.csv", index = False)

In [ ]:
pursuit_list = []
tmp = df["pursuit"].to_list()
for pursuits in tmp:
  for pursuit in pursuits.split(";"):
    pursuit_list.append(pursuit)

pursuit_count = pd.Series(pursuit_list).value_counts()
top_pursuit_list = list(pursuit_count[pursuit_count>10].index)
all_list = ["all"]
visit_list = ["visit", "travel"]
fashion_list = ["fashion", "makeup", "make up"]
dance_list = ["dance", "dancing"]
music_list = ["music", "sing"]
language_list = ["language", "speak"]

pursuit_df = pd.DataFrame()
pursuit_df["Num"] = df["Num"]
for pursuit in top_pursuit_list:
  if pursuit == "Learning about Korean/Asian culture":
    pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+language_list)) else 0)
  elif pursuit == "Learn about Korean fashion/makeup":
    pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+fashion_list)) else 0)
  elif pursuit == "Visit Korea":
    pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+visit_list)) else 0)
  elif pursuit == "Pursue dancing":
    pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+dance_list)) else 0)
  elif pursuit == "Learning music - either an instrument or singing":
    pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+music_list)) else 0)


def other_pursuit(x):
  check_list = []
  for tmp in x.split(";"):
    if all(keyword not in x for keyword in top_pursuit_list) and \
       all(keyword not in x.lower() for keyword in (all_list+visit_list+fashion_list+dance_list+music_list+language_list)):
      check_list.append(True)
    else:
      check_list.append(False)
  if any(check_list):
    return 1
  else:
    return 0

pursuit_df["Others"] = df["pursuit"].apply(lambda x: other_pursuit(x))

pursuit_df.to_csv("pursuit.csv", index = False)

In [ ]:
yes_list = ["yes", "spend", "takes up", "time consuming","does","most", "as much as", "most of the time"]
# no_list = ["no", "doesnâ€™t","wouldnâ€™t","doesn't"]
notanymore_list = ["not as much", "first got", "used to"]
notsure_list = ["depend", "not really", "sometime","not sure"]

def time_consume_yn(x):
    if any(keyword in x.lower() for keyword in yes_list):
      return "Yes"
    elif any(keyword in x.lower() for keyword in notanymore_list):
      return "Not as much as it used to"
    elif any(keyword in x.lower() for keyword in notsure_list):
      return "Not sure"
    else:
      return "No"

df["time_consume_yn"] = df["time_consume_yn"].apply(lambda x: time_consume_yn(x))
df.groupby(["time_consume_yn"]).count()

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,daily_music_hr,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,life_changing,pos_eff,merch_spent_yr,money_obtain,concert_yn,crazy_thing,age,domicile,job,gender
time_consume_yn,,,,,,,,,,,,,,,,,,,,,,
No,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116
Not as much as it used to,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Not sure,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37
Yes,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84,84


In [ ]:
bully_list = ["criticism", "poking", "fun"]
sleep_list = ["sleep"]
family_list = ["socialise"]
friend_list = ["new", "bond", "increase","introducing"]
love_list = ["love", "yourself", "myself", "relive", "peace"]
no_list = ["no", "hasnâ€™t"]

def lifechange(x):
  if any(keyword in x.lower() for keyword in bully_list):
    return "Led to you being cajoled (made fun of) by classmates and family"
  elif any(keyword in x.lower() for keyword in sleep_list):
    return "Reduced the amount of sleep you get"
  elif any(keyword in x.lower() for keyword in family_list):
    return "Reduced your time to socialise with your friends/partners/family"
  elif any(keyword in x.lower() for keyword in friend_list):
    return "Making new friends"
  elif any(keyword in x.lower() for keyword in love_list):
    return "Learn to love myself"
  elif any(keyword in x.lower() for keyword in no_list):
    return "No affect"
  else:
    return "Other"

df["life_changing"] = df["life_changing"].apply(lambda x: lifechange(x))

df.groupby(["life_changing"]).count()

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,daily_music_hr,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,time_consume_yn,pos_eff,merch_spent_yr,money_obtain,concert_yn,crazy_thing,age,domicile,job,gender
life_changing,,,,,,,,,,,,,,,,,,,,,,
Learn to love myself,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Led to you being cajoled (made fun of) by classmates and family,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
Making new friends,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
No affect,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
Other,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15
Reduced the amount of sleep you get,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71
Reduced your time to socialise with your friends/partners/family,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10


In [ ]:
poseff_list = []
tmp = df["pos_eff"].to_list()
for eff in tmp:
  poseff_list.append(eff)

eff_count = pd.Series(poseff_list).value_counts()
top_poseff_list = list(eff_count[eff_count >10].index)
all_list = ["both", "all", "everything"]
stress_list = ["anxiety", "stress", "depression", "hard", "weak"]
friend_list = ["friend"]
selflove_list = ["love", "myself"]

poseff_df = pd.DataFrame()
poseff_df["Num"] = df["Num"]
for eff in top_poseff_list:
  if eff == "Reduced stress/anxiety/depression":
    poseff_df[eff] = df["pos_eff"].apply(lambda x: 1 if (eff in x) or any(keyword in x.lower() for keyword in (all_list+stress_list)) else 0)
  else:
    poseff_df[eff] = df["pos_eff"].apply(lambda x: 1 if (eff in x) or any(keyword in x.lower() for keyword in (all_list+friend_list)) else 0)

poseff_df["Learn self love"] = df["pos_eff"].apply(lambda x: 1 if any(keyword in x.lower() for keyword in selflove_list) else 0)

poseff_df["Others"] = df["pos_eff"].apply(lambda x: 1 if all(keyword not in x for keyword in top_poseff_list) and all(keyword not in x.lower() for keyword in(all_list+stress_list+friend_list+selflove_list)) else 0)

poseff_df.to_csv("positive_effect.csv", index = False)

In [ ]:
canada_list = ["canada","ca"]
europe_list = ["finland", "germany", "france", "sweden", "european"]
asia_list = ["asia", "turkey"]
usa_list = ["usa"]
southamerica_list = ["latin america", "caribbean"]
aus_list = ["australia"]
nz_list = ["new zealand"]
africa_list = ["africa"]
uk_list = ["uk"]

def country(x):
  if any(keyword in x.lower() for keyword in canada_list):
    return "Canada"
  elif any(keyword in x.lower() for keyword in europe_list):
    return "Europe"
  elif any(keyword in x.lower() for keyword in asia_list):
    return "Asia"
  elif any(keyword in x.lower() for keyword in usa_list):
    return "USA"
  elif any(keyword in x.lower() for keyword in southamerica_list):
    return "South America"
  elif any(keyword in x.lower() for keyword in aus_list):
    return "Australia"
  elif any(keyword in x.lower() for keyword in nz_list):
    return "New Zealand"
  elif any(keyword in x.lower() for keyword in africa_list):
    return "South Africa"
  elif any(keyword in x.lower() for keyword in uk_list):
    return "UK"

df["domicile"] = df["domicile"].apply(lambda x: country(x))

df.groupby(["domicile"]).count()

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,daily_music_hr,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,time_consume_yn,life_changing,pos_eff,merch_spent_yr,money_obtain,concert_yn,crazy_thing,age,job,gender
domicile,,,,,,,,,,,,,,,,,,,,,,
Asia,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
Australia,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
Canada,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
Europe,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31
New Zealand,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
UK,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26
USA,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140,140


In [ ]:
nurse_list = ["nurse", "nursing", "caregiver"]
receptiosnist_list = ["receptionist"]
student_list = ["student"]

def job(x):
  if any(keyword in x.lower() for keyword in nurse_list):
    return "Nurse/Caregiver"
  elif any(keyword in x.lower() for keyword in student_list):
    return "Student"
  else:
    return "Other"
  
df["job"] = df["job"].apply(lambda x: job(x))

df.groupby(["job"]).count()

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,daily_music_hr,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,time_consume_yn,life_changing,pos_eff,merch_spent_yr,money_obtain,concert_yn,crazy_thing,age,domicile,gender
job,,,,,,,,,,,,,,,,,,,,,,
Nurse/Caregiver,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Other,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26
Student,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210,210


In [ ]:
crz_list = []
tmp = df["crazy_thing"].to_list()
for crzth in tmp:
  for crz in crzth.split(";"):
    crz_list.append(crz)
crz_count = pd.Series(crz_list).value_counts()
top_crz_list = list(crz_count[crz_count>10].index)
sleep_list = ["sleep", "waking", "stayed up really late"]

df["crazy_thing"] = df["crazy_thing"].replace("Listened to K-Pop without studying or working despite having a deadline;I do online school and watching kpop videos keeps me from doing any work at all.", 
                                              "Listened to K-Pop without studying or working despite having a deadline")
df["crazy_thing"] = df["crazy_thing"].replace("Listened to K-Pop without studying or working despite having a deadline;Again, because of my own personal problems with procrastination, not necessarily because of K-Pop itself.",
                                              "Listened to K-Pop without studying or working despite having a deadline")
df["crazy_thing"] = df["crazy_thing"].replace("Listened to K-Pop without studying or working despite having a deadline;Missed days of work and school to attend concerts;Traveled to another state for a kpop concert",
                                              "Listened to K-Pop without studying or working despite having a deadline")
df["crazy_thing"] = df["crazy_thing"].replace("Missed days of work and school to attend concerts;missed school to ticket for bts","Missed days of work and school to attend concert")
df["crazy_thing"] = df["crazy_thing"].replace("Nothing as such; I mean, I almost skipped a day of school for Monsta X's comeback showcase that one time but I didn't.","Nothing as such")

crazything_df = pd.DataFrame()
crazything_df["Num"] = df["Num"]
for crz in top_crz_list:
  crazything_df[crz] = df["crazy_thing"].apply(lambda x: 1 if crz in x else 0)

crazything_df["Reduced amount of sleep due to watching award show or comebacks"] = df["crazy_thing"].apply(lambda x: 1 if any(keyword in x.lower() for keyword in sleep_list) else 0)
crazything_df.groupby(["Missed days of work and school to attend concerts"]).count()
crazything_df.to_csv("crazy_thing.csv", index = False)

In [ ]:
spent_list = []
tmp = df["merch_spent_yr"].to_list()
for merchspent in tmp:
  spent_list.append(merchspent)
spent_count = pd.Series(spent_list).value_counts()
top_spent_list = list(spent_count[spent_count>10].index)

spent1_list = ["below 50","less than $50","below 30$","10?"]
spent2_list = ["50 - 100"]
spent3_list = ["100 - 200"]
spent4_list = ["200-300","$300","$200"]
spent5_list = ["1k","$2,000","$5,000"]
spent6_list = ["depends","xmas","donâ€™t buy everything","donâ€™t even want to know","i've only ever bought"]
nospent_list = ["i don't spend anything on merchandise","donâ€™t have the money", "don't have money","gifts"]

def money_spent(x):
  if any(keyword in x.lower() for keyword in spent1_list):
    return "Below 50$"
  elif any(keyword in x.lower() for keyword in spent2_list):
    return "50-100$"
  elif any(keyword in x.lower() for keyword in spent3_list):
    return "100-200$"
  elif any(keyword in x.lower() for keyword in spent4_list):
    return "200-300$"
  elif any(keyword in x.lower() for keyword in spent5_list):
    return "Above 1000$"
  elif any(keyword in x.lower() for keyword in spent6_list):
    return "Depends through out time"
  elif any(keyword in x.lower() for keyword in nospent_list):
    return "I don't spend anything on merchandise"

df["merch_spent_yr"] = df["merch_spent_yr"].apply(lambda x: money_spent(x))

df.groupby(["merch_spent_yr"]).count()

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,daily_music_hr,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,time_consume_yn,life_changing,pos_eff,money_obtain,concert_yn,crazy_thing,age,domicile,job,gender
merch_spent_yr,,,,,,,,,,,,,,,,,,,,,,
100-200$,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
200-300$,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
50-100$,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57
Above 1000$,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Below 50$,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94
Depends through out time,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
I don't spend anything on merchandise,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52


In [ ]:
df

,Num,fav_group,pop_by_country,reason,first_year,kpop_gender_pref,daily_music_hr,watch_mv_yn,daily_mv_hr,obsessed_yn,kpop_news,pursuit,time_consume_yn,life_changing,pos_eff,merch_spent_yr,money_obtain,concert_yn,crazy_thing,age,domicile,job,gender
0,0,BTS,Its gaining popularity,Its different from the usual music. I like som...,1-2 years ago,"Its about the music, not the gender",5 or more,Yes,2-3 hours,Not sure,Subscribe to K-Pop news sites;Subscribe to You...,Learning about Korean/Asian culture;Pursue dan...,No,Led to you being cajoled (made fun of) by clas...,Reduced stress/anxiety/depression,50-100$,Have full-time job,Yes,Missed days of work and school to attend concerts,15 - 20 years,UK,Other,Female
1,1,BTS,Its gaining popularity,Its different from the usual music. I like som...,3-4 years ago,I listen to both,2-4 hours,Yes,2-3 hours,Yes,Subscribe to K-Pop news sites;Subscribe to You...,Learning music - either an instrument or singi...,Yes,Led to you being cajoled (made fun of) by clas...,Made friends who are like-minded,Below 50$,Working part-time & Doing chores,"I want to, but the K-Pop groups don't come to ...",Listened to K-Pop without studying or working ...,15 - 20 years,UK,Student,Female
2,2,BTS;MONSTA X,Its gaining popularity,Its different from the usual music. I like som...,3-4 years ago,"Its about the music, not the gender",2-4 hours,Yes,2-3 hours,No,Subscribe to K-Pop news sites;Join K-pop socia...,Learning about Korean/Asian culture;Learn abou...,No,Led to you being cajoled (made fun of) by clas...,Made friends who are like-minded,50-100$,Working part-time & Doing chores,Yes,Nothing as such,15 - 20 years,Europe,Student,Female
3,3,BTS,Its gaining popularity,Its different from the usual music. I like som...,3-4 years ago,I listen to both,5 or more,Yes,3-4 hours,No,Subscribe to K-Pop news sites;Subscribe to You...,Learning about Korean/Asian culture;Visit Korea,No,Led to you being cajoled (made fun of) by clas...,Reduced stress/anxiety/depression,50-100$,Working part-time & Doing chores,Yes,Nothing as such,21 - 26 years,USA,Student,Female
4,4,BTS,Yes,Its different from the usual music. I like som...,3-4 years ago,Yes,5 or more,Yes,2-3 hours,Not sure,Subscribe to Youtube channels,Learning about Korean/Asian culture,Yes,Led to you being cajoled (made fun of) by clas...,Reduced stress/anxiety/depression,100-200$,Parents,"I want to, but the K-Pop groups don't come to ...",Nothing as such,15 - 20 years,USA,Student,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,235,MONSTA X,Its gaining popularity,Its different from the usual music. I like som...,3-4 years ago,I listen to both,5 or more,Yes,3-4 hours,Not sure,Subscribe to Youtube channels,Pursue dancing;Learn about Korean fashion/makeup,Not sure,Reduced the amount of sleep you get,Reduced stress/anxiety/depression,I don't spend anything on merchandise,-,"I want to, but the K-Pop groups don't come to ...",Nothing as such,15 - 20 years,Canada,Student,Female
236,236,BTS,Yes,Its different from the usual music. I like som...,3-4 years ago,Yes,2-4 hours,No,1 hour,No,Twitter,Learn about Korean fashion/makeup,No,Reduced your time to socialise with your frien...,Made friends who are like-minded,Above 1000$,Working part-time & Doing chores,Yes,Missed days of work and school to attend conce...,15 - 20 years,USA,Student,Female
237,237,BTS;GOT 7,I'm not sure,The videos are unique,Less than a year ago,Yes,5 or more,Yes,2-3 hours,Yes,Subscribe to Youtube channels;Join K-pop socia...,Learning about Korean/Asian culture;Learn abou...,Yes,Reduced the amount of sleep you get,Reduced stress/anxiety/depression,I don't spend anything on merchandise,-,No,Listened to K-Pop without studying or working ...,10 - 14 years,USA,Student,Female
238,238,BTS,Yes,Its different from the usual music. I like som...,3-4 years ago,I listen to both,2-4 hours,Yes,2-3 hours,Not sure,Subscribe to Youtube channels;Join K-pop socia...,Learning about Korean/Asi

In [ ]:
df = df.drop(columns=['fav_group', 'reason', 'kpop_news', 'pursuit', 'pos_eff', 'crazy_thing'])
df.to_csv("kpop_data_modified.csv", index = False)